In [1]:
import torch
from NetMF import NetMF
from Netmfutils import Netmfutils
import scipy.io
import numpy as np

In [2]:
if torch.cuda.is_available():
        torch.cuda.set_device(0)

import logging
from cogdl.datasets import build_dataset
logger = logging.getLogger(__name__)
netutils = Netmfutils()

Using backend: pytorch


In [3]:
Label_repo = "dataset/blogcatalog.mat"
Embedding_repo = "embeddings"
Rank = 128
Dim = 128
Window = 1
Large_netmf = False
Rand_Seed = 1
Neg_sampling = 1

Start_train_ratio = 10
Stop_train_ratio = 90
Num_train_ratio = 9
C = 1
Num_split = 10

In [4]:
data = scipy.io.loadmat(Label_repo)
print (type(data['network']))
logger.info("Loading label from %s...", Label_repo)
label = netutils.load_label(file=Label_repo, variable_name="group")
logger.info("Label loaded!")

<class 'scipy.sparse.csc.csc_matrix'>
(10312, 39) <class 'numpy.ndarray'> 0 1


In [5]:
netmf = NetMF(Dim,Window,Rank,Neg_sampling,Large_netmf)
embedding = netmf.train(data['network'])

Running NetMF for a small window size...
Compute matrix %d-th power 1


In [6]:
train_ratios = np.linspace(Start_train_ratio,Stop_train_ratio,
            Num_train_ratio)
for tr in train_ratios:
        netutils.node_classification_loss(embedding, label, train_ratio=tr/100.,
                n_splits=Num_split, C=C, random_state=Rand_Seed)

(1031,) (9281,)
(1031,) (9281,)
(1031,) (9281,)
(1031,) (9281,)
(1031,) (9281,)
(1031,) (9281,)
(1031,) (9281,)
(1031,) (9281,)
(1031,) (9281,)
(1031,) (9281,)
%d fold validation, training ratio %f 10 0.1
Average micro %.2f, Average macro %.2f 32.918275631306756 14.587759929638116
(2062,) (8250,)
(2062,) (8250,)
(2062,) (8250,)
(2062,) (8250,)
(2062,) (8250,)
(2062,) (8250,)
(2062,) (8250,)
(2062,) (8250,)
(2062,) (8250,)
(2062,) (8250,)
%d fold validation, training ratio %f 10 0.2
Average micro %.2f, Average macro %.2f 35.211611107004174 17.110123504947566
(3093,) (7219,)
(3093,) (7219,)
(3093,) (7219,)
(3093,) (7219,)
(3093,) (7219,)
(3093,) (7219,)
(3093,) (7219,)
(3093,) (7219,)
(3093,) (7219,)
(3093,) (7219,)
%d fold validation, training ratio %f 10 0.3
Average micro %.2f, Average macro %.2f 36.37756577249311 18.46588138227881
(4124,) (6188,)
(4124,) (6188,)
(4124,) (6188,)
(4124,) (6188,)
(4124,) (6188,)
(4124,) (6188,)
(4124,) (6188,)
(4124,) (6188,)
(4124,) (6188,)
(4124,) (618

/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1492: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1492: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1492: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1492: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_divisio